https://inf.u-szeged.hu/~rfarkas/ML20/osztalyozas.html

I watched through StatQuest's Linear + Logistic Regression videos

The following material will be in general about classification as I strayed from the original course I followed

In [3]:
import pandas as pd
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=["age", "workclass", "fnlwgt", "education", "education-num",
           "marital-status", "occupation", "relationship", "race", "sex",
           "capital-gain", "capital-loss", "hours-per-week", "country", "salary"
           ]
    )
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
classlabel = df.salary
features = df.iloc[:,:14]  # az első 14 jellemző alapján történik az előrejelzés

In [10]:
features

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


## Decision Trees

In [23]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()

In [25]:
# a döntési fa tanítása a fit() függvénnyel a jellemzővektorok és osztálycímkék (célváltozó) alapján
dt.fit(features, classlabel)

ValueError: could not convert string to float: ' State-gov'

It must be continous!  Therefore let's convert it <br>
Ordinal Encorder -> every discrete variable gets a random ordinal

In [42]:

from sklearn import preprocessing
e = preprocessing.OrdinalEncoder()


e.fit(features) #put our features in to the encorder
e.categories_ #categories of the encoder

[array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 90], dtype=int64),
 array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype=object),
 array([  12285,   13769,   14878, ..., 1366120, 1455435, 1484705],
       dtype=int64),
 array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
        ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
        ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
        ' Some-college'], dtype=object),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=int64),
 array([' Divorced', ' Married-AF-spouse', ' Marrie

In [44]:
enc_features = e.transform(features) #converts categorical features into their encoded form
enc_features

array([[2.2000e+01, 7.0000e+00, 2.6710e+03, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01],
       [3.3000e+01, 6.0000e+00, 2.9260e+03, ..., 0.0000e+00, 1.2000e+01,
        3.9000e+01],
       [2.1000e+01, 4.0000e+00, 1.4086e+04, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01],
       ...,
       [4.1000e+01, 4.0000e+00, 7.8830e+03, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01],
       [5.0000e+00, 4.0000e+00, 1.2881e+04, ..., 0.0000e+00, 1.9000e+01,
        3.9000e+01],
       [3.5000e+01, 5.0000e+00, 1.7825e+04, ..., 0.0000e+00, 3.9000e+01,
        3.9000e+01]])

In [46]:
enc_df = pd.DataFrame(enc_features, columns=features.columns) #make a new dataframe from the encoded data, with the original column names
enc_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country
0,22.0,7.0,2671.0,9.0,12.0,4.0,1.0,1.0,4.0,1.0,25.0,0.0,39.0,39.0
1,33.0,6.0,2926.0,9.0,12.0,2.0,4.0,0.0,4.0,1.0,0.0,0.0,12.0,39.0
2,21.0,4.0,14086.0,11.0,8.0,0.0,6.0,1.0,4.0,1.0,0.0,0.0,39.0,39.0
3,36.0,4.0,15336.0,1.0,6.0,2.0,6.0,0.0,2.0,1.0,0.0,0.0,39.0,39.0
4,11.0,4.0,19355.0,9.0,12.0,2.0,10.0,5.0,2.0,0.0,0.0,0.0,39.0,5.0


In [48]:
features.age.value_counts()

age
36    898
31    888
34    886
23    877
35    876
     ... 
83      6
88      3
85      3
86      1
87      1
Name: count, Length: 73, dtype: int64

In [50]:
enc_df.age.value_counts()

age
19.0    898
14.0    888
17.0    886
6.0     877
18.0    876
       ... 
66.0      6
71.0      3
68.0      3
69.0      1
70.0      1
Name: count, Length: 73, dtype: int64

In [52]:
features.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
dtype: object

In [58]:
e = preprocessing.OrdinalEncoder()
cat_features = features.select_dtypes(include=['object']) # categorized columns that are object type
enc_features = e.fit_transform(cat_features)
enc_features = pd.DataFrame(enc_features, columns=cat_features.columns)
enc_features

,workclass,education,marital-status,occupation,relationship,race,sex,country
0,7.0,9.0,4.0,1.0,1.0,4.0,1.0,39.0
1,6.0,9.0,2.0,4.0,0.0,4.0,1.0,39.0
2,4.0,11.0,0.0,6.0,1.0,4.0,1.0,39.0
3,4.0,1.0,2.0,6.0,0.0,2.0,1.0,39.0
4,4.0,9.0,2.0,10.0,5.0,2.0,0.0,5.0
...,...,...,...,...,...,...,...,...
32556,4.0,7.0,2.0,13.0,5.0,4.0,0.0,39.0
32557,4.0,11.0,2.0,7.0,0.0,4.0,1.0,39.0
32558,4.0,11.0,6.0,1.0,4.0,4.0,0.0,39.0
32559,4.0,11.0,4.0,1.0,3.0,4.0,1.0,39.0


In [62]:
enc_features = pd.concat( [features.select_dtypes(exclude=['object']), enc_features], axis=1 ) # continous and categorized attributes concatenation

## One Hot Encoding

Another way to encode the values of a discrete variable is to add a new binary attribute for each value and for a concrete individual, exactly one new binary attribute value will be 1, assigned to the value of that attribute, the rest will be 0 (one hot encoding). **If there is no ordering between the values (and there are more than two values) then you need OneHot!** <br>

In a one hot encoded matrix, we have a lot of places with 0 values and only a few places with 1 values. For memory and runtime reasons, we don't store the whole matrix in a contiguous memory address, but only the non-0 elements (i.e. the list of position-value pairs). We call this a sparse matrix.<br>

We usually use this more! While it avoid misleading information that comes from  ordinal encoding, it can become a multi dimensional hell (incomplete note)

In [70]:
ohe = preprocessing.OneHotEncoder() #one hot encoding
ohe_features = ohe.fit_transform(features)
ohe_features # sparse matrix (only storing the non null elements)

<32561x22144 sparse matrix of type '<class 'numpy.float64'>'
	with 455854 stored elements in Compressed Sparse Row format>

In [74]:
# in readable format
pd.DataFrame.sparse.from_spmatrix(ohe_features, columns=ohe.get_feature_names_out(features.columns))

,age_17,age_18,age_19,age_20,age_21,age_22,age_23,age_24,age_25,age_26,...,country_ Portugal,country_ Puerto-Rico,country_ Scotland,country_ South,country_ Taiwan,country_ Thailand,country_ Trinadad&Tobago,country_ United-States,country_ Vietnam,country_ Yugoslavia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
32557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
32558,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
32559,0,0,0,0,0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0


## Building the Decision Tree

In [79]:
dt = tree.DecisionTreeClassifier(max_depth=10) # döntési fa paraméterezése, legfeljebb 3 mély fát építhet
dt.fit(ohe_features, classlabel) # teaching

DecisionTreeClassifier(max_depth=10)

In [81]:
!pip install eli5

     ---------------------------------------- 0.0/216.2 kB ? eta -:--:--
     - -------------------------------------- 10.2/216.2 kB ? eta -:--:--
     --- --------------------------------- 20.5/216.2 kB 330.3 kB/s eta 0:00:01
     ------- ----------------------------- 41.0/216.2 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 216.2/216.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/47.1 kB ? eta -:--:--
   ---------------------------------------- 47.1/47.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107762 sha256=7fee40d38b5b6439d790ad44a28f105ab452c89941651122401b2f8b1163efa2
  Stored in directory: c:\users\weebmachine\appdata\local\pip\cache\wheels\e0\e4\92\b2763717e9a525c427c049221a2fa8fa4d0ec6190bfc587fc8
Successfully built eli5


In [113]:
!pip install --upgrade scikit-learn

import eli5

   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.9 MB 640.0 kB/s eta 0:00:18
   - -------------------------------------- 0.3/10.9 MB 3.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/10.9 MB 6.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.9 MB 8.9 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/10.9 MB 10.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/10.9 MB 12.9 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/10.9 MB 15.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.4/10.9 MB 17.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.5/10.9 MB 21.0 MB/s eta 0:00:01
   ---------------------------------------  10.9/10.9 MB 31.2 MB/s eta 0:00:01
   ---------------------------------------- 10.9/10.9 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: t

  You can safely remove it manually.


ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (F:\Programs\Anaconda\Lib\site-packages\sklearn\utils\metaestimators.py)

In [87]:
classlabel.value_counts()

salary
<=50K    24720
>50K      7841
Name: count, dtype: int64

In [115]:
eli5.show_weights(dt, feature_names=list(ohe_features.columns), show=["decision_tree"], filled="True")
#this would show a decision tree but idk the error (prob installation error)

NameError: name 'eli5' is not defined

## Predictions

In [94]:
# az első 20K példát használjuk tanító adatbázisnak
train_features = ohe_features[:20000]
train_labels = classlabel[:20000]
# és csak azon tanítjuk a döntési fát
dt.fit(train_features, train_labels)

DecisionTreeClassifier(max_depth=10)

In [102]:
# majd a többi (20K. utáni) példát használjuk kiértékelő (test) adatbázisnak
test_features = ohe_features[20000:]
prediction = dt.predict(test_features)
prediction

array([' <=50K', ' >50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [100]:
# kiértékelési metrikának a találati arányt (accuracy) használjuk
from sklearn.metrics import accuracy_score
accuracy_score(prediction, classlabel[20000:]) # predikció és igazi osztálycímkék összevetése

0.8505692221956851

Mindig hasonlítsuk a gépi tanulás eredményét [baseline](http://inf.u-szeged.hu/~rfarkas/ML20/baseline.html) döntésekhez!



In [106]:

from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent") # tanító adatbázis leggyakoribb osztálya lesz mindig a predikció
dummy_clf.fit(train_features, train_labels) # ugyanazon a tanító adatbázison "tanítjuk"
baseline_prediction = dummy_clf.predict(test_features) # predikció a kiértékelő adatbázison
accuracy_score(baseline_prediction, classlabel[20000:])

0.7547965926279755